In [10]:
import pandas as pd
import requests

In [35]:
gw = 3

In [36]:
team_stats_new_season_for = pd.read_csv(f"C:/Users/erknud3/fpl-optimization/model/data/teams_new_season_for_gw{gw}.csv")
team_stats_new_season_against = pd.read_csv(f"C:/Users/erknud3/fpl-optimization/model/data/teams_new_season_against_gw{gw}.csv")

In [37]:
team_stats_new_season_for = team_stats_new_season_for[['team_id', 'Squad', '90s_new', 'weighted_npxG']]
team_stats_new_season_against = team_stats_new_season_against[['team_id', 'Squad', '90s_new', 'weighted_npxGC']]

In [38]:
r = requests.get("https://fantasy.premierleague.com/api/fixtures/")
fixtures = pd.DataFrame(r.json())
fixtures = fixtures[['event', 'team_h', 'team_a']]

In [40]:
avg_npxG = team_stats_new_season_for['weighted_npxG'].mean()
avg_npxGC = team_stats_new_season_against['weighted_npxGC'].mean()

In [41]:
# Step 1: Calculate and Round Attack and Defense Multipliers
team_stats_new_season_for['attack_multiplier'] = (team_stats_new_season_for['weighted_npxG'] / avg_npxG).round(2)
team_stats_new_season_against['defense_multiplier'] = (team_stats_new_season_against['weighted_npxGC'] / avg_npxGC).round(2)

# Step 2: Rename weighted_npxG and weighted_npxGC
team_stats_new_season_for.rename(columns={'weighted_npxG': 'baseline_npxG'}, inplace=True)
team_stats_new_season_against.rename(columns={'weighted_npxGC': 'baseline_npxGC'}, inplace=True)

# Step 3: Initialize New Columns
for gw in range(1, 39):
    team_stats_new_season_for[gw] = 0.0
    team_stats_new_season_against[gw] = 0.0

# Step 4: Process Fixtures and Populate Gameweek Columns
for index, row in fixtures.iterrows():
    gw = row['event']
    team_h = row['team_h']
    team_a = row['team_a']
    
    # Get attack and defense multipliers
    home_attack = team_stats_new_season_for.loc[team_stats_new_season_for['team_id'] == team_h, 'attack_multiplier'].values[0]
    away_attack = team_stats_new_season_for.loc[team_stats_new_season_for['team_id'] == team_a, 'attack_multiplier'].values[0]
    home_defense = team_stats_new_season_against.loc[team_stats_new_season_against['team_id'] == team_h, 'defense_multiplier'].values[0]
    away_defense = team_stats_new_season_against.loc[team_stats_new_season_against['team_id'] == team_a, 'defense_multiplier'].values[0]
    
    # Get baseline npxG and npxGC
    home_npxG = team_stats_new_season_for.loc[team_stats_new_season_for['team_id'] == team_h, 'baseline_npxG'].values[0]
    away_npxG = team_stats_new_season_for.loc[team_stats_new_season_for['team_id'] == team_a, 'baseline_npxG'].values[0]
    home_npxGC = team_stats_new_season_against.loc[team_stats_new_season_against['team_id'] == team_h, 'baseline_npxGC'].values[0]
    away_npxGC = team_stats_new_season_against.loc[team_stats_new_season_against['team_id'] == team_a, 'baseline_npxGC'].values[0]
    
    # Adjust for home/away and opponent strength, then round to 2 decimals
    team_stats_new_season_for.loc[team_stats_new_season_for['team_id'] == team_h, gw] = round(home_npxG * away_defense * 1.12, 2)
    team_stats_new_season_for.loc[team_stats_new_season_for['team_id'] == team_a, gw] = round(away_npxG * home_defense * 0.88, 2)
    
    team_stats_new_season_against.loc[team_stats_new_season_against['team_id'] == team_h, gw] = round(home_npxGC * away_attack * 0.88, 2)
    team_stats_new_season_against.loc[team_stats_new_season_against['team_id'] == team_a, gw] = round(away_npxGC * home_attack * 1.12, 2)

# Step 5: Drop the attack_multiplier and defense_multiplier columns
team_stats_new_season_for.drop(columns=['attack_multiplier'], inplace=True)
team_stats_new_season_against.drop(columns=['defense_multiplier'], inplace=True)

# Step 6: Ensure all columns are rounded to 2 decimals
team_stats_new_season_for = team_stats_new_season_for.round(2)
team_stats_new_season_against = team_stats_new_season_against.round(2)


In [44]:
max_mp = team_stats_new_season_for["90s_new"].max().astype(int)

csv_file_path = "C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data"

if not team_stats_new_season_for.empty:
    team_stats_new_season_for.to_csv(
        f"{csv_file_path}/teams_pred_npxG_gw{max_mp}.csv", index=False
    )
    print(f"'For' data successfully saved to teams_pred_npxG.csv")
else:
    print("No 'for' data to save.")
    
if not team_stats_new_season_against.empty:
    team_stats_new_season_against.to_csv(
        f"{csv_file_path}/teams_pred_npxGC_gw{max_mp}.csv", index=False
    )
    print(f"'For' data successfully saved to teams_pred_npxGC.csv")
else:
    print("No 'for' data to save.")

'For' data successfully saved to teams_new_season_for_gw.csv
'For' data successfully saved to teams_new_season_for_gw.csv


In [46]:
# Step 1: Calculate the multipliers for each gameweek in team_stats_new_season_for
for gw in range(1, 39):
    team_stats_new_season_for[gw] = (team_stats_new_season_for[gw] / team_stats_new_season_for['baseline_npxG']).round(2)

# The team_stats_new_season_for DataFrame now contains the multipliers for each gameweek.

In [47]:
team_stats_new_season_for

,team_id,Squad,90s_new,baseline_npxG,1,2,3,4,5,6,...,29,30,31,32,33,34,35,36,37,38
0,1,Arsenal,3.0,1.77,1.23,0.89,1.01,0.87,0.51,1.37,...,1.05,1.20,0.87,1.10,1.23,0.99,1.16,0.71,1.11,1.18
1,2,Aston Villa,3.0,1.49,0.98,0.64,1.07,1.11,1.23,1.22,...,0.91,0.79,1.13,1.18,1.11,0.51,1.20,0.91,1.11,0.96
2,3,Bournemouth,3.0,1.30,0.89,1.11,0.87,1.05,0.72,1.50,...,1.10,1.55,0.98,1.20,0.78,1.22,0.50,1.13,0.51,1.37
3,4,Brentford,3.0,1.43,1.00,0.71,1.50,0.51,0.87,1.24,...,0.92,0.87,1.06,0.50,1.01,0.89,1.22,1.22,1.20,0.97
4,5,Brighton,3.0,1.46,0.87,1.22,0.50,1.55,1.13,0.83,...,0.51,1.13,0.78,1.36,0.86,1.25,1.11,0.97,0.90,0.87
5,6,Chelsea,2.0,1.60,0.65,0.97,0.99,0.91,0.98,1.01,...,0.50,1.11,0.86,1.56,0.94,1.11,0.91,0.87,1.22,0.89
6,7,Crystal Palace,2.0,1.15,0.86,1.24,0.83,1.37,1.22,0.87,...,0.87,1.18,1.01,0.51,1.17,0.50,1.13,0.87,1.23,0.71
7,8,Everton,3.0,1.28,1.01,0.88,1.16,0.89,1.07,1.00,...,1.24,0.71,0.64,0.89,0.65,0.83,1.55,0.95,1.50,0.88
8,9,Fulham,3.0,1.22,0.96,1.37,1.22,1.25,1.11,0.89,...,1.11,0.50,0.91,0.92,1.05,1.18,0.89,1.11,0.86,0.65
9,10,Ipswich Town,3.0,0.77,0.91,0.51,1.19,0.79,1.18,1.13,...,1.13,0.91,1.23,0.83,0.64,0.87,0.87,1.10,1.08,1.25


In [49]:
from scipy.stats import poisson
import numpy as np

# Step 1: Initialize an empty DataFrame to store the probabilities
columns = ['team_id', 'Squad'] + [f'{gw}_{goals}_goals' for gw in range(1, 39) for goals in range(9)]
team_concede_probs = pd.DataFrame(columns=columns)

# Step 2: Calculate probabilities for each team and gameweek
for idx, row in team_stats_new_season_against.iterrows():
    team_id = row['team_id']
    squad = row['Squad']
    
    probabilities = []
    
    for gw in range(1, 39):
        mean_goals_conceded = row[gw]
        probs = poisson.pmf(range(9), mean_goals_conceded).round(4)
        probabilities.extend(probs)
    
    # Insert the data into the new DataFrame
    team_concede_probs.loc[idx] = [team_id, squad] + list(probabilities)

# The team_concede_probs DataFrame now contains the probabilities for conceding 0 to 8 or more goals for each gameweek.


In [56]:
# Initialize a new DataFrame to store the cumulative probabilities
GC_probabilities = pd.DataFrame(columns=['team_id', 'Squad'] + [f'{gw}_{concede}_goals' for gw in range(1, 39) for concede in [0, 1, 2, 4, 6, 8]])

# Calculate cumulative probabilities for each team and gameweek
for idx, row in team_concede_probs.iterrows():
    team_id = row['team_id']
    squad = row['Squad']
    
    cum_probs = []
    
    for gw in range(1, 39):
        prob_0 = row[f'{gw}_0_goals']
        prob_1 = row[f'{gw}_1_goals']
        prob_2 = row[f'{gw}_2_goals']
        prob_4 = row[f'{gw}_4_goals']
        prob_6 = row[f'{gw}_6_goals']
        prob_8_or_more = row[f'{gw}_8_goals'] + sum(row[f'{gw}_{k}_goals'] for k in range(9, len(row) // 38))
        
        cum_probs.extend([prob_0, prob_1, prob_2, prob_4, prob_6, prob_8_or_more])
    
    # Insert the data into the new DataFrame
    GC_probabilities.loc[idx] = [team_id, squad] + cum_probs

# The cumulative_probabilities DataFrame now contains probabilities for 0, 1, 2, 4, 6, and 8 or more goals.

In [58]:
csv_file_path = "C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data"

GC_probabilities.to_csv(f"{csv_file_path}/GC_probabilities.csv", index=False)